In [1]:
import uuid
from openpyxl.styles import Font, Fill, Border, Alignment, PatternFill
from copy import copy
import pandas as pd, openpyxl
import os

from column_mapping import column_mapping

template_file_path = os.path.abspath("workday_data.xlsx")
temp_file_path = os.path.abspath(f"{uuid.uuid4()}.xlsx")
print(template_file_path)

master_data_sheet_name="Masterdata_PSteam"
employee_master_sheet_name="Employee Master"
hiring_data_sheet_name="Hiring_data"
lineup_pass_import_sheet_name="LINEUP_PASS_IMPORT"
termination_sheet_name="Termination"

master_data_skip_rows = 11
employee_master_skip_rows=11
hiring_data_skip_rows=1
lineup_pass_import_skip_rows=0
termination_skip_rows=10

master_df = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                           sheet_name=master_data_sheet_name, skiprows=master_data_skip_rows)

employee_master = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                           sheet_name=employee_master_sheet_name, skiprows=employee_master_skip_rows)

hiring_data = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                           sheet_name=hiring_data_sheet_name,  skiprows=hiring_data_skip_rows)

lineup_pass_import = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                           sheet_name=lineup_pass_import_sheet_name,  skiprows=lineup_pass_import_skip_rows)
termination = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                            sheet_name=termination_sheet_name, skiprows=termination_skip_rows)



merged_data_first = pd.merge(
    hiring_data,
    employee_master,
    left_on='Emp id',
    right_on='EMPLOYEE_NUMBER',
    how='left'
)
merged_data_second = pd.merge(
    merged_data_first,
    lineup_pass_import,
    left_on='National ID (SSN/SIN) (National Identifiers)',
    right_on='Số CMND/CCCD:',
    how='left'
)
merged_df = pd.merge(
    merged_data_second,
    termination,
    left_on='Emp id',
    right_on='EMPLOYEE_ID',
    how='left'
)
try:
    result_df = master_df.merge(merged_df, left_on='EID', right_on='Emp id', how='outer', suffixes=('_master', '_merged'))

    # Hàm hợp nhất các cột trùng lặp
    def combine_columns(df, source_cols, target_col):
        cols = [col for col in df.columns if col in source_cols]
        if cols:
            df[target_col] = df[cols].bfill(axis=1).iloc[:, 0]
            df.drop(columns=cols, inplace=True)
        return df

    # Áp dụng ánh xạ và hợp nhất cột
    for target_col, source_col in [(k, v) for v, k in column_mapping.items()]:
        source_cols = [col for col in master_df.columns if col in column_mapping and column_mapping[col] == target_col]
        if source_cols:
            master_df = combine_columns(master_df, source_cols, target_col)

    # Loại bỏ các cột không nằm trong master_data.xlsx
    master_data = master_df.dropna().loc[:, ~master_df.columns.str.contains('^Unnamed')]
    master_columns = master_data.columns.tolist()
    valid_columns = master_columns + list(column_mapping.values())
    result_df = result_df[[col for col in result_df.columns if col in valid_columns]]

    # Lưu kết quả
    result_df.to_excel('final_merged_output.xlsx', index=False)

    # wb = openpyxl.load_workbook(template_file_path)
    # ws = wb[master_data_sheet_name]
    #
    # # Lấy style của dòng header (dòng master_data_skip_rows + 1 trong Excel)
    # header_row = master_data_skip_rows + 1
    # header_styles = []
    # for cell in ws[header_row]:
    #     style = {
    #         'font': copy(cell.font) if cell.font else Font(),
    #         'fill': copy(cell.fill) if cell.fill else PatternFill(),
    #         'border': copy(cell.border) if cell.border else Border(),
    #         'alignment': copy(cell.alignment) if cell.alignment else Alignment()
    #     }
    #     header_styles.append(style)
    #
    # # Mở file đầu ra để áp dụng style
    # wb_output = openpyxl.load_workbook(output_file)
    # ws_output = wb_output['master_data']
    #
    # # Áp dụng style cho dòng header (dòng 1 trong file đầu ra)
    # for col_idx, style in enumerate(header_styles, start=1):
    #     cell = ws_output.cell(row=1, column=col_idx)
    #     cell.font = style['font']
    #     cell.fill = style['fill']
    #     cell.border = style['border']
    #     cell.alignment = style['alignment']
    #
    # # Lưu file đầu ra
    # wb_output.save(output_file)
except Exception as e:
    print(e)

C:\Users\ManageITVN\PyCharmMiscProject\workday_data.xlsx


C:\Users\ManageITVN\AppData\Local\Temp\ipykernel_25200\2387085930.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[target_col] = df[cols].bfill(axis=1).iloc[:, 0]
C:\Users\ManageITVN\AppData\Local\Temp\ipykernel_25200\2387085930.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[target_col] = df[cols].bfill(axis=1).iloc[:, 0]
